In [1]:
import pandas as pd 
import copy
import numpy as np
import datetime as datetime
import json
file_name = '20250120_MORPHE2US.xlsx'

In [2]:
from source.Node import Node
from source.Unit import Unit
from source.Municipality import Municipality
from source.District import District
from source.Building import Building
from source.Model import Model, Temporal_block, Report
from source.Connection import Connection
from source.Storage import Storage

In [3]:
## GENERAL COMMODITIES IN THE DICTIONARY ## 
df_commodities = pd.read_excel(file_name, sheet_name='Commodities', header=1)
commodities_names = [col for col in df_commodities.columns if not col.startswith("Unnamed") and not col.startswith("Name") and not col.startswith("code")]
df_commodities_NaM = df_commodities.set_index("code")
dict__general_nodes = {}
dict__mandatory_units = {}
for commodity in commodities_names:
    new_node = Node()
    new_node.set_name(commodity)
    
    if "NaM_overflow_lost" in df_commodities_NaM.index:
        if df_commodities_NaM.loc["NaM_overflow_lost", commodity] == True:
            print(f"Overflow {commodity}")
            new_node.add_direct_parameter('NaM_overflow_lost', True, 'boolean')
            unit = Unit()
            unit.set_name(f"Overflow_{commodity}")
            unit.add_direct_parameter("NaM_unit__from_node1", commodity, 'string')
            dict__mandatory_units[f"Overflow_{commodity}"] = unit
    if "NaM_balance_type" in df_commodities_NaM.index:
        if df_commodities_NaM.loc["NaM_balance_type", commodity] == True:
            print(f"Balance type None for {commodity}")
            new_node.add_direct_parameter('balance_type', "balance_type_none", 'string')
    dict__general_nodes[commodity] = new_node

new_node = Node()
new_node.set_name('CO2')
dict__general_nodes['CO2'] = new_node

Overflow Heat
Balance type None for Heating Oil


In [4]:
## GENERAL UNITS IN THE DICTIONARY ##
df_units = pd.read_excel(file_name, sheet_name='Units', header=1)
df_units = df_units.loc[:, ~df_units.columns.str.contains('^Unnamed')]
nb_units = df_units.shape[1] - 3


df_units = pd.read_excel(file_name, sheet_name= 'Units')
dict__general_units = {}
for i in range(nb_units):
    df_unit = df_units.iloc[:, [1, 2, i+3]]
    df_unit = df_unit.dropna()
    new_unit = Unit()
    for index, row in df_unit.iterrows():
        type_ = row.iloc[0]
        tech_name = row.iloc[1]
        value = row.iloc[2]
        new_unit.add_direct_parameter(tech_name, value, type_)
    new_unit.add_co2()
    dict__general_units[new_unit.get_name()] = new_unit

In [5]:
## GENERAL STORAGES IN THE DICTIONARY ##
df_storages = pd.read_excel(file_name, sheet_name='Storages', header=1)
df_storages = df_storages.loc[:, ~df_storages.columns.str.contains('^Unnamed')]
nb_storages = df_storages.shape[1] - 3


df_storages = pd.read_excel(file_name, sheet_name= 'Storages')
dict__general_storages = {}
for i in range(nb_storages):
    df_storage = df_storages.iloc[:, [1, 2, i+3]]
    df_storage = df_storage.dropna()
    new_storage = Storage()
    for index, row in df_storage.iterrows():
        type_ = row.iloc[0]
        tech_name = row.iloc[1]
        value = row.iloc[2]
        new_storage.add_direct_parameter(tech_name, value, type_)
    dict__general_storages[new_storage.get_name()] = new_storage

In [6]:
## GENERAL BUILDING TYPES IN THE DICTIONARY ##

df_buildings = pd.read_excel(file_name, sheet_name='Building types', header=1)
df_buildings = df_buildings.loc[:, ~df_buildings.columns.str.contains('^Unnamed')]
nb_buildings = df_buildings.shape[1] - 1

dict__general_building_types = {}

for i in range(nb_buildings):
    df_building = df_buildings.iloc[:, [0, i+1]]
    type_ = df_building.iloc[0, 1]
    construction_year = df_building.iloc[1, 1]
    new_building = Building(str(df_building.columns[1]), type_, construction_year)
    df_retrofits = df_building.loc[df_building[df_building["name"]== "Retrofits"].index[0]:]
    indexes = df_retrofits[df_retrofits["name"] == "Commodity"].index

    for index in indexes:
        df_retrofit = df_retrofits.loc[index-1:index+3].dropna()
        if df_retrofit.empty:
            break
        name = df_retrofit.iloc[0, 1]
        commodity_to_invest = df_retrofit.iloc[1, 1]
        retrofit_increase_performance = df_retrofit.iloc[2, 1]
        retrofit_cost = df_retrofit.iloc[3, 1]
        new_building.add_retrofit(name, commodity_to_invest, retrofit_increase_performance, retrofit_cost)
    
    for key in dict__general_nodes.keys():
        new_building.add_node(copy.deepcopy(dict__general_nodes[key]))
    for key in dict__mandatory_units.keys():
        new_building.add_unit(copy.deepcopy(dict__mandatory_units[key]))

    dict__general_building_types[new_building.name] = new_building

In [7]:
## GENERAL CONNECTIONS TYPES IN THE DICTIONARY ##	

df_connections = pd.read_excel(file_name, sheet_name='Connections', header=1)
df_connections = df_connections.loc[:, ~df_connections.columns.str.contains('^Unnamed')]
nb_connections = df_connections.shape[1] - 3
df_connections = pd.read_excel(file_name, sheet_name='Connections')
dict__general_connections = {}
for i in range(nb_connections):
    df_connection = df_connections.iloc[:, [1, 2, i+3]]
    df_connection = df_connection.dropna()
    new_connection = Connection()
    for index, row in df_connection.iterrows():
        type_ = row.iloc[0]
        tech_name = row.iloc[1]
        value = row.iloc[2]
        new_connection.add_direct_parameter(tech_name, value, type_)
    dict__general_connections[new_connection.get_name()] = new_connection

In [8]:
## CREATING THE MUNICIPALITY AND THE DISTRICTS WITHIN IT ##

municipality = Municipality("Sourcieux")
df_districts = pd.read_excel(file_name, sheet_name='Districts', header=1)
districts_names = [col for col in df_districts.columns if not col.startswith("Unnamed") and not col.startswith("Name")]

start = None
list_district = []
for i, district in enumerate(districts_names):
    for index, col in enumerate(df_districts.columns):
        # Allocate parts of df_districts to each district in a list of dataframes
        if col == districts_names[i]:
            start = index
            if i == len(districts_names) - 1:
                list_index = [1] + list(range(start, df_districts.shape[1])) 
                list_district.append(df_districts.iloc[:, list_index])
                break
        if start != None and col == districts_names[i+1]:
            end = index
            list_index = [1] + list(range(start, end)) 
            list_district.append(df_districts.iloc[:, list_index])
            start = None
for index, district in enumerate(list_district):
    list_district[index] = district.dropna(subset=["Name"])

In [9]:
## FILLING THE UNITS & STORAGES WITHIN THE DISTRICT AT DISTRICT LEVEL ## 
def extract_and_add_entities(df_district, district, idx0, idx1, dict__general):
    first_idx = df_district[df_district["Name"] ==idx0].index[0]
    if idx1 == -1:
        df_district_entity = df_district.loc[(first_idx+1):, :]
    else: 
        last_idx = df_district[df_district["Name"] == idx1].index[0]
        df_district_entity = df_district.loc[(first_idx+1):(last_idx-1), :]
    for j, row in df_district_entity.iterrows():
        entity_name = row.iloc[0]
        number_of_units = 0 if np.isnan(row.iloc[1]) else int(row.iloc[1])
        candidate_units = 0 if np.isnan(row.iloc[2]) else int( row.iloc[2])
        new_entity = copy.deepcopy(dict__general[entity_name])
        new_entity.add_direct_parameter("number_of_units", number_of_units)
        if not(candidate_units == 0):
            new_entity.add_direct_parameter("candidate_units", candidate_units)
        if not(number_of_units == 0 and candidate_units == 0):
            if isinstance(new_entity, Storage):
                district.add_storage(new_entity)
            if isinstance(new_entity, Unit):
                district.add_unit(new_entity)
    return district



for i in range(len(list_district)):
    df_district = list_district[i]
    new_district = District(districts_names[i])

    # Build all the nodes at district level
    for key in dict__general_nodes.keys():
        new_district.add_node(copy.deepcopy(dict__general_nodes[key]))
    for key in dict__mandatory_units.keys():
        new_district.add_unit(copy.deepcopy(dict__mandatory_units[key]))

    new_district = extract_and_add_entities(df_district, new_district,  "Units presence (district level)", "Units presence (building level)", dict__general_units)
    new_district = extract_and_add_entities(df_district, new_district,  "Storages presence (district level)", "Storages presence (building level)", dict__general_storages)


    columns_buildings = list(df_district.iloc[0,:])
    df_district_building = df_district.loc[df_district["Name"].isin(["Building", "Quantity"])].dropna(axis=1).iloc[:, 1:].T
    df_district_building = df_district_building.reset_index()

    for k, row in df_district_building.iterrows():
        building_name = row.iloc[1]
        building_quantity = row.iloc[2]
        
        if building_quantity == 0:
            continue
        new_building = copy.deepcopy(dict__general_building_types[building_name])
        new_building.set_quantity(building_quantity)
        df_building_district_unit = (df_district.iloc[:, [0, columns_buildings.index(building_name),  columns_buildings.index(building_name) + 1]])

        new_building = extract_and_add_entities(df_building_district_unit, new_building, "Units presence (building level)", "Storages presence (district level)", dict__general_units)
        new_building = extract_and_add_entities(df_building_district_unit, new_building, "Storages presence (building level)", -1, dict__general_storages)

        new_district.add_building(new_building)
    municipality.add_district(new_district)

In [10]:
## Add the exporting/importing units ## 

df_connections = pd.read_excel(file_name, sheet_name='Commodities', header=1)

for commodity_name in commodities_names:
    df_connection_commodity = df_connections.iloc[:, [2, df_connections.columns.get_loc(commodity_name)]]
    df_connection_commodity = df_connection_commodity.set_index(df_connection_commodity.columns[0])

    if "NaM_exp_dis" in df_connection_commodity.index \
    and type(df_connection_commodity.loc["NaM_exp_dis"][commodity_name]) == str:
        print("Add export unit")
        export_unit = Unit()
        export_unit.add_direct_parameter("name", f"export_{commodity_name}")
        export_unit.add_direct_parameter("NaM_unit__from_node1", commodity_name)
        exp_cap = df_connection_commodity.loc["NaM_exp_cap"][commodity_name]
        if not np.isnan(exp_cap):
            export_unit.add_direct_parameter("unit_capacity(unit__from_node1)", exp_cap)
        export_unit.add_direct_parameter("NaM_emission", 0) ### Change it according to carbon intensity of the commodity
        for district in municipality.districts:
            if district.get_name() in df_connection_commodity.loc["NaM_exp_dis"][commodity_name]:
                district.add_unit(copy.deepcopy(export_unit))

    if "NaM_imp_dis" in df_connection_commodity.index \
    and type(df_connection_commodity.loc["NaM_imp_dis"][commodity_name]) == str:
        print("Add import unit")
        import_unit = Unit()
        import_unit.add_direct_parameter("name", f"import_{commodity_name}")
        import_unit.add_direct_parameter("NaM_unit__to_node1", commodity_name)
        imp_cap = df_connection_commodity.loc["NaM_imp_cap"][commodity_name]
        if not np.isnan(imp_cap):
            import_unit.add_direct_parameter("unit_capacity(unit__to_node1)", imp_cap)
        import_unit.add_direct_parameter("NaM_emission", 0) ### Change it according to carbon intensity of the commodity
        for district in municipality.districts:
            if district.get_name() in df_connection_commodity.loc["NaM_imp_dis"][commodity_name]:
                district.add_unit(copy.deepcopy(import_unit))

In [11]:
## Build the connections between the districts and within the district ##

df_connections = pd.read_excel(file_name, sheet_name='Commodities', header=1)
indexes_connections = df_connections[df_connections["Name"] == "Connection"].index
for commodity_name in commodities_names:
    df_connections_commodity = df_connections.iloc[:, [0, 1, 2, df_connections.columns.get_loc(commodity_name)]]
    
    for i, index in enumerate(indexes_connections):
        
        if i == len(indexes_connections) - 1:
            df_connection_commodity = df_connections_commodity.loc[(index+1):, :].dropna(subset=[commodity_name])
        else:
            df_connection_commodity = df_connections_commodity.loc[(index+1): indexes_connections[i+1]-2, :].dropna(subset=[commodity_name])
        if df_connection_commodity.empty:
            continue
        
        connection_name = [row.iloc[3] for index_row, row in df_connection_commodity.iterrows() if row.iloc[2] == "name"][0]
        connection = dict__general_connections[connection_name]
        for index_row, row in df_connection_commodity.iterrows():
            if row.iloc[2] == "candidate_connections" and float(row.iloc[3]) == 0:
                continue
            connection.add_direct_parameter(row.iloc[2], float(row.iloc[3]) if row.iloc[1] == "float" else row.iloc[3], row.iloc[1])


        district_from  = connection.direct_parameters["NaM_district_lvl(from_node)"]["value"] if "NaM_district_lvl(from_node)" in connection.direct_parameters.keys() else None
        building_from = connection.direct_parameters["NaM_building_lvl(from_node)"]["value"] if "NaM_building_lvl(from_node)" in connection.direct_parameters.keys() else None
        district_to  = connection.direct_parameters["NaM_district_lvl(to_node)"]["value"] if "NaM_district_lvl(to_node)" in connection.direct_parameters.keys() else None
        building_to = connection.direct_parameters["NaM_building_lvl(to_node)"]["value"] if "NaM_building_lvl(to_node)" in connection.direct_parameters.keys() else None

        if district_from != None and building_from == None and district_to != None and building_to == None:
            # District to district connection: Connection stored in the 
            municipality.add_district_interconnection(connection, commodity_name, district_from, district_to)

        if district_from != None and building_from == None and district_to != None and building_to != None:
            # District to building connection: Connection stored in the district (same district from and to !!)
            if district_from != district_to: 
                print("Error in the building to district connection: It should be the same district")
            for district in municipality.districts:
                if district.get_name() in district_from:
                    district.add_building_connection(connection, commodity_name, building_to, flag_direction_building = "to")
                
        if district_from != None and building_from != None and district_to != None and building_to == None:
            # print(f"Building to distrit connection: {connection.name} {commodity_name} {district_from},{building_from} {district_to}")
            if district_from != district_to: 
                print("Error in the building to district connection: It should be the same district")
            for district in municipality.districts:
                # Building to district: Connection stored in the district
                if district.get_name() in district_to:
                    district.add_building_connection(connection, commodity_name, building_from, flag_direction_building = "from")


In [12]:
# Building the reports in the model 

df_specs = pd.read_excel(file_name, sheet_name='Specifications')
df_specs_model = df_specs.iloc[df_specs[df_specs["Model"] == "General"].index[0]: df_specs[df_specs["Model"] == "Operation"].index[0], :].dropna(subset=["code", "value"])
model = Model()

for j, row in df_specs_model.iterrows():
    model.add_direct_parameter(row.iloc[3], row.iloc[4], row.iloc[2]) # code = row.iloc[3], value = row.iloc[4], type_ = row.iloc[2]


df_reports = pd.read_excel(file_name, sheet_name='Reports')
df_reports = df_reports.loc[:, ~df_reports.columns.str.contains('^Unnamed')]
nb_reports = df_reports.shape[1] - 3 # 3 first columns are Name, Description and code

for i in range(nb_reports):
    df_report = df_reports.iloc[:, [2, i+3]]
    report = Report(df_report.columns[1])
    for index, row in df_report.iterrows():
        if row.iloc[1] == True:
            report.add_output(row.iloc[0])
    if report.get_output_list_length() > 0:
        model.add_report(report)

In [13]:
## Building the operations of the model 
df_specs_operation = df_specs.iloc[df_specs[df_specs["Model"] == "Operation"].index[0]: df_specs[df_specs["Model"] == "Economic"].index[0], :].dropna(subset=["code", "value"])

if (list(df_specs_operation[df_specs_operation["code"] == "NaM_linear_op"].value))[0] == True:
    df_specs_operation = df_specs.iloc[(df_specs[df_specs["code"] == "NaM_linear_op"].index[0]+1): df_specs[df_specs["code"] == "NaM_bool_specific_year"].index[0], :].dropna(subset=["code", "value"])
    print(df_specs_operation)
    operation = Temporal_block()
    for j, row in df_specs_operation.iterrows():
        operation.add_direct_parameter(row.iloc[3], row.iloc[4], row.iloc[2]) # code = row.iloc[3], value = row.iloc[4], type_ = row.iloc[2]
    operation.add_direct_parameter("block_start", model.direct_parameters["model_start"]["value"], "datetime")
    operation.add_direct_parameter("block_end", model.direct_parameters["model_end"]["value"], "datetime")
    model.add_operation(operation)
elif (list(df_specs_operation[df_specs_operation["code"] == "NaM_bool_specific_year"].value))[0] == True:
    df_specs_operation = df_specs.iloc[(df_specs[df_specs["code"] == "NaM_bool_specific_year"].index[0]+1): df_specs[df_specs["code"] == "NaM_Representative_days"].index[0], :].dropna(subset=["code", "value"])
    specific_years = (list(df_specs_operation[df_specs_operation["code"] == "NaM_specific_year"].value))[0]
    for year in specific_years.split(";"):
        year = int(year)
        start_date = datetime.datetime(year, 1, 1, 0, 0, 0)
        end_date = datetime.datetime(year+1, 1, 1, 0, 0, 0)
        model_start = model.direct_parameters["model_start"]["value"]
        model_end = model.direct_parameters["model_end"]["value"]
        operation = Temporal_block()
        if year > model_end.year or year < model_start.year:
            continue
        if start_date <model_start:
            operation.add_direct_parameter("block_start",model_start, "datetime")
        else:
            operation.add_direct_parameter("block_start", start_date, "datetime")
        if end_date > model_end:
            operation.add_direct_parameter("block_end", model_end, "datetime")
        else:
            operation.add_direct_parameter("block_end", end_date, "datetime")
        operation.add_direct_parameter("resolution", (list(df_specs_operation[df_specs_operation["code"] == "resolution"].value))[0], "duration")
        operation.add_direct_parameter("name", f'{(list(df_specs_operation[df_specs_operation["code"] == "name"].value))[0]}_{year}', "duration")
        model.add_operation(operation)
elif (list(df_specs_operation[df_specs_operation["code"] == "NaM_Representative_days"].value))[0] == True:
    print("Representative days to be implemented")


In [14]:
# Building the investments of the model

df_specs = pd.read_excel(file_name, sheet_name='Specifications')
df_specs_investment = df_specs.iloc[df_specs[df_specs["Model"] == "Economic"].index[0]:df_specs[df_specs["Model"] == "CO2"].index[0], :].dropna(subset=["code", "value"])
df_specs_investment = df_specs_investment.loc[:, ["type","code", "value"]]
df_specs_investment.reset_index(drop=True, inplace=True)
if (list(df_specs_investment[df_specs_investment["code"] == "NaM_invstmt_bool"].value))[0] == True:
    if (list(df_specs_investment[df_specs_investment["code"] == "NaM_invstmt_sgl_bool"].value))[0] == True:
        df_specs_investment_sgl = df_specs_investment.iloc[df_specs_investment[df_specs_investment["code"] == "NaM_invstmt_sgl_bool"].index[0]+1:df_specs_investment[df_specs_investment["code"] == "NaM_invstmt_multi_bool"].index[0], :]
        investment = Temporal_block()
        for j, row in df_specs_investment_sgl.iterrows():
            investment.add_direct_parameter(row.iloc[1], row.iloc[2], row.iloc[0]) 
        investment.add_direct_parameter("block_end", model.direct_parameters["model_end"]["value"], "datetime")
        investment.add_direct_parameter("resolution", "100Y", "duration")
        model.add_investment(investment)
    if (list(df_specs_investment[df_specs_investment["code"] == "NaM_invstmt_multi_bool"].value))[0] == True:
        df_specs_investment_multi = df_specs_investment.iloc[df_specs_investment[df_specs_investment["code"] == "NaM_invstmt_multi_bool"].index[0]+1:, :]
        investment = Temporal_block()
        for j, row in df_specs_investment_multi.iterrows():
            investment.add_direct_parameter(row.iloc[1], row.iloc[2], row.iloc[0])
        investment.add_direct_parameter("block_end", model.direct_parameters["model_end"], "datetime")
        model.add_investment(investment)

In [15]:
# Building the CO2 node and connections within the municipality

df_specs = pd.read_excel(file_name, sheet_name='Specifications')
df_specs_co2 = df_specs.iloc[df_specs[df_specs["Model"] == "CO2"].index[0]:, :].dropna(subset=["code", "value"])
df_specs_co2 = df_specs_co2.loc[:, ["type","code", "value"]]
df_specs_co2.reset_index(drop=True, inplace=True)
if (list(df_specs_co2[df_specs_co2["code"] == "NaM_CO2_bool"].value))[0] == True:
    df_specs_co2 = df_specs_co2.iloc[df_specs_co2[df_specs_co2["code"] == "NaM_CO2_bool"].index[0]+1:, :]
    if (list(df_specs_co2[df_specs_co2["code"] == "NaM_CO2_state_lvl"].value))[0] == "Building":
        node_CO2 = Node()
        node_CO2.add_direct_parameter("has_state", True, "boolean")
        node_CO2.add_direct_parameter("initial_node_state", 0, "float")
        if len(list(df_specs_co2[df_specs_co2["code"] == "node_state_cap"].value)) > 0:
            node_capacity = list(df_specs_co2[df_specs_co2["code"] == "node_state_cap"].value)[0]
            node_CO2.add_direct_parameter("node_state_cap", node_capacity, "float")
        node_CO2.add_direct_parameter("name", "CO2")
        CO2_connection = Connection()
        CO2_connection.add_direct_parameter("connection_type", "connection_type_lossless_bidirectional")
        CO2_connection.add_direct_parameter("name", "Connection_CO2_M-LVL")
        # Not necessary as the capacity is unlimited if not specified
        # CO2_connection.add_direct_parameter("connection_capacity(to_node)", 10000)
        municipality.add_node(node_CO2)
        municipality.add_CO2_connection(CO2_connection, "CO2")
else:
    for district in municipality.districts:
        for node in district.nodes:
            if "CO2" in node.get_name():
                node.add_direct_parameter("balance_type", "balance_type_none", "string")
        for building in district.buildings:
            for node in building.nodes:
                if "CO2" in node.get_name():
                    node.add_direct_parameter("balance_type", "balance_type_none", "string")


In [16]:
# Building the vectors/time series and assigning it to the correct units/nodes/connections
df_datalist =  pd.read_excel(file_name, sheet_name='Vectors_datalist')

indexes_datalists = [i for i, x in enumerate(df_datalist.iloc[0]) if type(x) == str]
datalist = []
type_value = ""
for index_af in indexes_datalists[1:]:
    df_af = df_datalist.iloc[:, [0, index_af, index_af+1]]
    df_af = df_af.dropna(how='all')
    datalist_parameter = {}
    for index, row in df_af.iterrows():
        if type(row.iloc[0]) != str:
            break
        datalist_parameter[row.iloc[0]] = row.iloc[1]
    
    if datalist_parameter["Time Serie type"] == "Fixed Resolution":
        value = [{"start": datalist_parameter["Start date"].strftime('%Y-%m-%dT%H:%M:%S'), "resolution": datalist_parameter["Resolution"],
                  "ignore_year": datalist_parameter["Ignore year"], "repeat": datalist_parameter["Repeat"]},
                  list(df_af.iloc[index+1:,2])]
        type_value = "time_series"
    elif datalist_parameter["Time Serie type"] == "Variable Resolution":
        time_vector = list(df_af.iloc[index+1:, 1])
        data_dict_temp = {}
        value_vector = list(df_af.iloc[index+1:, 2])
        for i, time in enumerate(time_vector):
            data_dict_temp[time.strftime('%Y-%m-%dT%H:%M:%S')] = value_vector[i]
        value = [{"ignore_year": datalist_parameter["Ignore year"], "repeat": datalist_parameter["Repeat"]},
                 data_dict_temp]
        type_value = "time_series"
    elif datalist_parameter["Time Serie type"] == "Single value":
        value = df_af.iloc[index+1, 2]
        type_value = "float"
    
    datalist.append([datalist_parameter, value, type_value])

import json
with open('demands\\demand.json') as f:
    demands = json.load(f)

for demand in demands:
    datalist.append(demand)
    

for datavector in datalist:
    
    if datavector[0]["Type"] == "Availability Factor":
        municipality.add_availability_factor(datavector[0]["District"], datavector[0]["Building"], datavector[0]["Unit"], datavector[1], datavector[2])

    # Local demand #
    elif datavector[0]["Type"] == "Local demand":
        try: 
            municipality.add_local_demand(datavector[0]["Commodity"], datavector[0]["District"], datavector[0]["Building"], datavector[1], datavector[2])
        except:
            print(datavector)
    
    # Export price #
    elif datavector[0]["Type"] == "Export price":
        for district in municipality.districts:
            if datavector[0]["District"] == "All" or district.get_name() == datavector[0]["District"]:
                for unit in district.units:
                    if type(datavector[0]["Unit"]) == str:  # Case of a specific unit
                        if datavector[0]["Unit"] in unit.get_name():
                            unit.add_direct_parameter("vom_cost(unit__from_node1)", datavector[1], datavector[2])
                    else:
                        if f"export_{datavector[0]["Commodity"]}" in unit.get_name():
                            unit.add_direct_parameter("vom_cost(unit__from_node1)", datavector[1], datavector[2])

    # Import price #
    elif datavector[0]["Type"] == "Import price":        
        for district in municipality.districts:
            if datavector[0]["District"] == "All" or district.get_name() == datavector[0]["District"]:
                for unit in district.units:
                    if type(datavector[0]["Unit"]) == str:  # Case of a specific unit
                        if datavector[0]["Unit"] in unit.get_name():
                            unit.add_direct_parameter("vom_cost(unit__to_node1)", datavector[1], datavector[2])
                    elif "balance_type" in dict__general_nodes[datavector[0]["Commodity"]].direct_parameters.keys():
                        for key in unit.direct_parameters.keys():
                            if key.startswith("NaM_unit__from_node") and unit.direct_parameters[key]["value"] == datavector[0]["Commodity"]:
                                unit.add_direct_parameter(f"vom_cost({key[4:]})", datavector[1], datavector[2])
                                break
                    else:
                        if f"import_{datavector[0]["Commodity"]}" in unit.get_name():
                            unit.add_direct_parameter("vom_cost(unit__to_node1)", datavector[1], datavector[2])
                for building in district.buildings:
                    for unit in building.units:
                        if type(datavector[0]["Unit"]) == str:  # Case of a specific unit
                            if datavector[0]["Unit"] in unit.get_name():
                                unit.add_direct_parameter("vom_cost(unit__to_node1)", datavector[1], datavector[2])
                        elif "balance_type" in dict__general_nodes[datavector[0]["Commodity"]].direct_parameters.keys():
                            for key in unit.direct_parameters.keys():
                                if key.startswith("NaM_unit__from_node") and unit.direct_parameters[key]["value"] == datavector[0]["Commodity"]:
                                    unit.add_direct_parameter(f"vom_cost({key[4:]})", datavector[1], datavector[2])
                                    break
                        else:
                            if f"import_{datavector[0]["Commodity"]}" in unit.get_name():
                                unit.add_direct_parameter("vom_cost(unit__to_node1)", datavector[1], datavector[2])

c:\Users\thoma\AppData\Local\Programs\Python\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [17]:
# Add the retrofit mode 
for district in municipality.districts:
    for building in district.buildings:
        building.create_building_retrofit_mode()

In [18]:
model.__dict__

{'operation_list': [],
 'investment_list': [],
 'modelisation_structure': [],
 'report_list': [Report(name='Report_1', output_list=['connection_flow', 'connections_invested', 'node_state', 'storages_invested', 'unit_flow', 'units_invested', 'units_on'])],
 'direct_parameters': {'model_start': {'value': datetime.datetime(2025, 1, 1, 0, 0),
   'type': 'datetime'},
  'model_end': {'value': datetime.datetime(2033, 1, 1, 0, 0),
   'type': 'datetime'},
  'use_economic_representation': {'value': False, 'type': 'bool'},
  'use_milestone_years': {'value': False, 'type': 'bool'}},
 'name': 'simple_model',
 'location_name': ''}

In [19]:
with open('template_files/template_new.json') as f:
    data_template = json.load(f)

model.add_modelisation_structure(municipality)
data_template = model.export_json(data_template)

with open('output.json', 'w') as f:
    json.dump(data_template, f, indent = 4)

In [20]:
# add Multiyear ()
# add multi scenario
# add possibility to add MAP
